In [1]:
import pandas as pd
import numpy as np
import shutil
import subprocess
import time
import pandas as pd
import sqlite3
import numpy as np
calibration_start_time = time.time()


# MinDist = 1 #1~3
# MaxAcc = 3 #2.5~3
# NormalDec = 4 #4~5.3
# MaxDec = 6 #5~9.3
# MinHeadway = 0 #0.25~1.25
# SensitivityFactor = 1 #0~1


def newParameter(initial_solution):
    Parameter = np.genfromtxt('DrivingBehaviorParameter.csv', delimiter=',')
    Parameter = initial_solution
    np.savetxt('DrivingBehaviorParameter.csv', Parameter, delimiter=',', fmt='%s')

   
    # cmd1 = '"C:\\Program Files\\Aimsun\\Aimsun Next 23\\Aimsun Next.exe" --project "D:\\RealTwin\\Real-Twin Workflow Paper\\aimsun simulation_0506actuatedsignal\\chatt.ang" --command execute --target 1269'
    cmd1 = '"C:\\Program Files\\Aimsun\\Aimsun Next 23\\aconsole.exe" -script "C:\\Users\\xugua\\Downloads\\RealTwin Calibration\\aimsun simulation_0506actuatedsignal\\DrivingBehavior.py" "C:\\Users\\xugua\\Downloads\\RealTwin Calibration\\aimsun simulation_0506actuatedsignal\\chatt.ang"'
    process = subprocess.Popen(cmd1, shell=True)
    # process = subprocess.Popen(cmd1, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    process.wait()

def runAimsun():
    cmd2 = '"C:\\Program Files\\Aimsun\\Aimsun Next 23\\aconsole.exe" --project "C:\\Users\\xugua\\Downloads\\RealTwin Calibration\\aimsun simulation_0506actuatedsignal\\chatt.ang" --command execute --target 955'
    process = subprocess.Popen(cmd2, shell=True)
    process.wait()

def resultAnalysis():
    dataname = "Resources\Outputs\chatt.sqlite"
    con = sqlite3.connect(dataname)
    section0 = pd.read_sql_query("SELECT * from MISECT", con)
    section = section0[(section0['sid'] == 1) & (section0['ent'] == 1)]
    con.close()
    # section.to_excel('section.xlsx', index=False)
    compare = pd.read_excel('compare.xlsx')
    compare = compare.merge(section[['oid', 'count']], left_on='entrance', right_on='oid', how='left')
    compare.drop(columns='oid', inplace=True)
    compare['GEH'] = np.sqrt(2 * ((compare['count'] - compare['realcount']) ** 2) / (compare['count'] + compare['realcount']))
    meanGEH = compare['GEH'].mean()
    GEHPercent = (compare['GEH'] < 5).mean()
    return meanGEH,GEHPercent

def resultFitness(path1_id,path2_id,path1_tt,path2_tt):
    dataname = "Resources\Outputs\chatt.sqlite"
    con = sqlite3.connect(dataname)
    subpath0 = pd.read_sql_query("SELECT * from MISUBPATH", con)
    tt1 = subpath0[(subpath0['oid'] == path1_id) & (subpath0['sid'] == 0) & (subpath0['ent'] == 0)]['ttime'].iloc[0]
    tt2 = subpath0[(subpath0['oid'] == path2_id) & (subpath0['sid'] == 0) & (subpath0['ent'] == 0)]['ttime'].iloc[0]
    con.close()
    MAE = (abs(path1_tt-tt1)+abs(path2_tt-tt2))/2
    
    return MAE

In [ ]:
# GA Parameters
MinDist = 1 #1~3
MaxAcc = 2.5 #2.5~3
NormalDec = 4 #4~5.3
MaxDec = 5 #5~9.3
MinHeadway = 0.25 #0.25~1.25
SensitivityFactor = 1 #0~1
#HeadwayAggressiveness=1 #-1~1
#AccAggressiveness=1 #-1~1

path1_id = 1286 #EB
path2_id = 1287 #WB
path1_tt = 240
path2_tt = 180

parameter_ranges = [
    (1, 3),   # Range for min dist/spacing
    (2.5, 3),   # Range for max acceleration
    (4, 5.3), # Range for normal deceleration
    (5, 9.3),    # Range for max deceleration
    (0.25, 1.25), # Range for min headway
    (0, 1)   # Range for sensitivity factor
]

# initial_solution = np.array([1,3,4,6,0,1])




iterations = 30
tabu_size = 120
neighborhood_size = 32
move_range = 0.5  # Initial move range
max_no_improvement_local = 5
max_no_improvement_global = 30
lbp = 0
ubp = 1
lbc = 0
ubc = 100
Np = len(parameter_ranges)




def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

def denormalize(normalized_value, min_val, max_val):
    return normalized_value * (max_val - min_val) + min_val


def objective_function(x,path1_id,path2_id,path1_tt,path2_tt,parameter_ranges):
    # Denormalize the parameters
    denormalized_x = np.zeros_like(x)
    for i in range(len(x)):
        min_val, max_val = parameter_ranges[i]
        denormalized_x[i] = denormalize(x[i], min_val, max_val)
    newParameter(denormalized_x)
    runAimsun()
    value = resultFitness(path1_id,path2_id,path1_tt,path2_tt)
    return value




def is_close_to_tabu(solution, tabu_list,Np, thresholdp,thresholdc):
    flagp = 0
    flagc = 0
    for tabu_solution in tabu_list:
        if np.linalg.norm(solution[0:Np] - tabu_solution[0:Np]) < thresholdp:
            flagp = 1

    for tabu_solution in tabu_list:
        if np.linalg.norm(solution[Np:] - tabu_solution[Np:]) < thresholdc:
            flagc = 1
    
    return flagp*flagc



num_variables = len(parameter_ranges)
initial_params_notnormalized = np.zeros(num_variables)
for i in range(num_variables):
    min_val, max_val = parameter_ranges[i]
    initial_params_notnormalized[i] = np.random.uniform(min_val, max_val)
initial_params = np.zeros_like(initial_params_notnormalized)
for i in range(num_variables):
    min_val, max_val = parameter_ranges[i]
    initial_params[i] = normalize(initial_params_notnormalized[i], min_val, max_val)



best_solution = initial_params.copy()
best_value = objective_function(best_solution,path1_id,path2_id,path1_tt,path2_tt,parameter_ranges)


prev_value = best_value
tabu_list = [initial_params.tolist()]
MAE_summary = []
solution_summary = []
print('Initial MAE is {}.'.format(best_value))
failcount = 0
for iter in range(iterations):
    print('Calibration iteration {}:'.format(iter))
    flag_move = 0
    iloc = 0
    solution_iter = []
    while (flag_move ==0 and iloc < max_no_improvement_local):
        iloc +=1
        print('        Calibration iteration {}, local search trail {}:'.format(iter,iloc))
        GEH_iter = []
        for neighbor in range(neighborhood_size):
            neighborhood = best_solution.copy()
            
            while True:   # continue generate if close to tabu list
                for i in range(len(neighborhood)):
                    while True:
                        real_minus_range = move_range
                        real_plus_range = move_range     
                        move = np.random.uniform(-real_minus_range,real_plus_range)
                        if i<=Np-1:
                            if lbp <= neighborhood[i] + move <= ubp:
                                neighborhood[i] += move
                                break 
                        else:
                            if lbc <= neighborhood[i] + move*ubc <= ubc:
                                neighborhood[i] += move
                                break 
                
                if is_close_to_tabu(neighborhood, tabu_list,Np, 0.05,10) == 0:
                    break             
                # if any(neighborhood.tolist() == tabu_sol for tabu_sol in tabu_list):
                #     break  
            # print(neighborhood)  
              
            neighborhood_value = objective_function(neighborhood,path1_id,path2_id,path1_tt,path2_tt,parameter_ranges)
            print('                Neighbour {}: MAE is {}.'.format(neighbor,neighborhood_value))
            GEH_iter.append(neighborhood_value)
            solution_iter.append(neighborhood)
            if neighborhood_value < best_value:
                print('better')
                best_solution, best_value = neighborhood.copy(), neighborhood_value
                flag_move = 1

    if flag_move == 1:
        print('  Successfully make an improvement at iteration {}.'.format(iter))
        # move_range -=0.05
        move_range *=0.9
        tabu_list.append(best_solution.tolist())
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)
        failcount = 0
    else:
        # move_range +=0.05
        move_range *=1.1
        failcount +=1
        print('  No improvement at iteration {}. Have failed to improve for {} iterations'.format(iter,failcount))
    solution_summary.append(solution_iter)
    MAE_summary.append(GEH_iter)
    print('Current best mean MAE is {}.'.format(best_value))
    print("Current calibration time is {} sec.".format(time.time() - calibration_start_time))
    if failcount >= max_no_improvement_global:
        print("No improvement is made in the last {} iterations. Calibration will be terminated.".format(max_no_improvement_global))
        break
        


# Find the best solution
denormalized_best_solution = np.zeros_like(best_solution)
for i in range(len(best_solution)):
    min_val, max_val = parameter_ranges[i]
    denormalized_best_solution[i] = denormalize(best_solution[i], min_val, max_val)
newParameter(denormalized_best_solution)
runAimsun()
best_MAE = resultFitness(path1_id,path2_id,path1_tt,path2_tt)
final_GEH,final_GEHpercent = resultAnalysis()
print("Best solution:", denormalized_best_solution)
print("Best MAE:", best_MAE) 
print("Final GEH:", final_GEH) 
print("In final results, {} percent GEH is lower than 5.".format(int(final_GEHpercent*10000)/100))

np.savetxt('TS_parameter_best.txt', denormalized_best_solution , fmt='%f')



Initial MAE is 47.78777122956494.
Calibration iteration 0:
        Calibration iteration 0, local search trail 1:
                Neighbour 0: MAE is 49.03876012121263.
                Neighbour 1: MAE is 33.11611117523762.
better
                Neighbour 2: MAE is 57.37615835575453.
                Neighbour 3: MAE is 43.636124223517314.
                Neighbour 4: MAE is 52.20485199781568.
                Neighbour 5: MAE is 41.29404559274515.
                Neighbour 6: MAE is 43.50275166189661.
                Neighbour 7: MAE is 45.807607484985226.
                Neighbour 8: MAE is 42.57108739927695.
                Neighbour 9: MAE is 39.720440408458686.
                Neighbour 10: MAE is 39.456813595984784.
                Neighbour 11: MAE is 51.70303943459028.
                Neighbour 12: MAE is 50.44648138824034.
                Neighbour 13: MAE is 42.517727079354174.
                Neighbour 14: MAE is 44.38335390519153.
                Neighbour 15: MAE is 38.3647

KeyboardInterrupt: 